In [6]:
import numpy as np
import pandas as pd

In [7]:
## get prices of Dow Jones constituents
index_prices = pd.read_csv('./indu_const_prices.csv', index_col=0, parse_dates=[0])

## get prices of index
price_data = pd.read_csv('./indu_prices.csv', index_col=0, parse_dates=[0])

returns = price_data.pct_change()[1:]
returns_clean = returns.dropna(axis=1)
price_data = price_data.loc[returns_clean.index]
corr_mat = np.corrcoef(returns_clean.T)

In [8]:
from bqplot import pyplot as plt
from bqplot import GridHeatMap, Figure, ColorScale, ColorAxis, HeatMap

from ipywidgets import HBox

In [9]:
def event_handler(name, value):
    sel = price_line.selected
    if sel is None or len(sel) < 2:
        _corr = np.corrcoef(returns_clean).T
    else:
        _corr = np.corrcoef(returns_clean.iloc[sel[0]:sel[-1], :]).T
    corr_mat.color = _corr

In [11]:
sc_col = ColorScale(scheme='Reds', min=0.0, max=1.0)
ax_col = ColorAxis(scale=sc_col, label='Correlation')

corr_mat = HeatMap(color=np.corrcoef(returns_clean.T),
                       scales={'color': sc_col})
corr_fig = Figure(marks=[corr_mat], axes=[ax_col], title='DOW Jones Correlation Matrix')

plt.figure(title='DOW Jones IA Price Series')
price_line = plt.plot(index_prices.index.values, index_prices.values)
brush = plt.int_selector(event_handler)
price_fig = plt.current_figure()
plt.axes()['x'].tick_format = '%b-%Y'
plt.axes()['x'].num_ticks = 5

HBox([price_fig, corr_fig])